In [1]:
import os
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
## Download required NLTK resources
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_

True

# Checking Dataset Structure

In [3]:
dataset_path="aclImdb"   

for root,dirs,files in os.walk(dataset_path):
    print(f"{root} contains {len(files)} files")
    if len(dirs) == 0:
        print(files[:5]) #show first 5 files in each folder
    break

aclImdb contains 3 files


# Reading the IMDB review into pandas

In [4]:
import pandas as pd
import glob

def load_imdb_data(directory,label):
    data=[]
    for file_path in glob.glob(f"{directory}/*.txt"):
        with open(file_path,"r",encoding="utf-8") as file:
            data.append((file.read(),label))
    return data

#upadate the path to to extracted dataset location
dataset_path="aclImdb"

## Loading positive and negative reviews

In [5]:
train_pos= load_imdb_data(f"{dataset_path}/train/pos","positive")
train_neg=load_imdb_data(f"{dataset_path}/train/neg","negative")
test_pos=load_imdb_data(f"{dataset_path}/test/pos","positive")
test_neg=load_imdb_data(f"{dataset_path}/test/neg","negative")

# Convert to pandas dataframe
train_df=pd.DataFrame(train_pos+train_neg,columns=["review","sentiment"])
test_df=pd.DataFrame(test_pos,test_neg,columns=["review","sentiment"])


In [6]:
print("Trainng data:")
print(train_df.head())

print("testing data:")
print(test_df.head())



Trainng data:
                                              review sentiment
0  Bromwell High is a cartoon comedy. It ran at t...  positive
1  Homelessness (or Houselessness as George Carli...  positive
2  Brilliant over-acting by Lesley Ann Warren. Be...  positive
3  This is easily the most underrated film inn th...  positive
4  This is not the typical Mel Brooks film. It wa...  positive
testing data:
                                                                                               review  \
(Once again Mr. Costner has dragged out a movie...  I went and saw this movie last night after bei...   
(This is an example of why the majority of acti...  Actor turned director Bill Paxton follows up h...   
(First of all I hate those moronic rappers, who...  As a recreational golfer with some knowledge o...   
(Not even the Beatles could write songs everyon...  I saw this film in a sneak preview, and it is ...   
(Brass pictures (movies is not a fitting word f...  Bill Paxton has t

In [13]:
## Initilize Lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [ ]:
# Function for text cleaning
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = re.sub(r"\d+", "", text)  # Remove numbers
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Remove stopwords & lemmatize
    return " ".join(tokens)


In [12]:
#applying preprocessing to reviews
train_df["cleaned_review"]=train_df["review"].apply(clean_text)
test_df["cleaned_review"]=test_df["review"].apply(clean_text)

#show cleaned data
print(train_df[["review","cleaned_review"]].head())

AttributeError: module 'string' has no attribute 'punctuatio'

# Convert text to features usning TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000) #limit features to 5000 for efficiency

#fit&transform text data
X_train = vectorizer.fit_transform(train_df["cleaned_review"]) 
X_test = vectorizer.transform(test_df["cleaned_review"])

#Encode labels (Convert "positive"-> 1 and negative -> 0)
from sklearn.preprocessing import LabelEncoder
encoder = LebelEncoder()
y_train=encoder.fit_transform(train_df)